In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Activation, add

In [ ]:
import random

samples = []
labels = []
images_folder = "images/"
image_list = os.listdir(images_folder)
random.seed(10)
random.shuffle(image_list)

for image in image_list:
    samples.append(img_to_array(load_img(images_folder + image, target_size = (200, 200)))) # Change Target Size accordingly to the resolution.
    if "normal" in image:
        labels.append((0))
    else:
        labels.append((1))
        
samples = np.array(samples)
labels = np.array(labels)
eval_x, eval_y = samples[-100:], labels[-100:] # 1000 images each for Normal and Soundscape, 100 in evaluation.
print(samples.shape, labels.shape)

x_train, x_test, y_train, y_test = train_test_split(samples[:-100], labels[:-100], test_size = 0.2, random_state=31)

In [ ]:
stride = 1
CHANNEL_AXIS = 3

def res_layer(x, filters, pooling = False, dropout = 0.0):
    temp = x
    temp = Conv2D(filters, (3,3), strides = stride, padding = "same")(temp)
    temp = BatchNormalization(axis = CHANNEL_AXIS)(temp)
    temp = Activation("relu")(temp)
    temp = Conv2D(filters, (3,3), strides = stride, padding = "same")(temp)

    x = add([temp, Conv2D(filters, (3,3), strides = stride, padding = "same")(x)])
    if pooling:
        x = MaxPooling2D((2,2))(x)
    if dropout != 0.0:
        x = Dropout(dropout)(x)
    x = BatchNormalization(axis = CHANNEL_AXIS)(x)
    x = Activation("relu")(x)
    return x

In [ ]:
%%time

from keras.callbacks import History 

inp = Input(shape = (200,200,3))

x = inp
x = Conv2D(16,(3,3),strides = stride,padding = "same")(x)
x = BatchNormalization(axis = CHANNEL_AXIS)(x)
x = Activation("relu")(x)
x = res_layer(x, 16, dropout = 0.2)
x = res_layer(x, 16, dropout = 0.3)
x = res_layer(x, 16, dropout = 0.4, pooling = True)
x = res_layer(x, 32, dropout = 0.2)
x = res_layer(x, 32, dropout = 0.2, pooling = True)
x = res_layer(x, 64, dropout = 0.4)
x = Flatten()(x)
x = Dropout(0.4)(x)

x = Dense(64,activation = "relu")(x)
x = Dropout(0.23)(x)
x = Dense(1,activation = "sigmoid")(x)

resnet_model = Model(inp,x,name = "Resnet")


resnet_model.summary()

resnet_model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
history = History()
resnet_model.fit(x_train, y_train, batch_size = 32, epochs = 50, callbacks = [history], validation_data = (x_test, y_test), verbose = 1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the training loss and accuracy
def plotResults(): 
    plt.figure()
    N = 50
    
    # TODO: plot the accuracy/loss variables over training time
    plt.plot(np.arange(0, N), history.history["accuracy"], label = "train_acc")
    plt.plot(np.arange(0, N), history.history["val_accuracy"], label = "val_acc")

    # make the graph understandable: 
    plt.title("Training Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.yscale('log')
    plt.legend(loc="upper right")
    plt.show()
    
    plt.figure()
    plt.plot(np.arange(0, N), history.history["loss"], label = "train_loss")
    plt.plot(np.arange(0, N), history.history["val_loss"], label = "val_loss")
    
     # make the graph understandable: 
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
resnet_model.save_weights('resnet-model.h5')
plotResults()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

def evaluate_on_training_set(y_test, y_pred):
    
    # Calculate AUC
    print("AUC  is:  ",  roc_auc_score(y_test,  y_pred))
    
    # print out recall and precision
    print(classification_report(y_test, y_pred))
    
    # print out confusion matrix
    print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
    
    # # calculate points for ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    
    # Plot ROC curve
    plt.plot(fpr,  tpr,  label='ROC  curve  (area  =  %0.3f)'  %  roc_auc_score(y_test,  y_pred)) 
    plt.plot([0,  1],  [0,  1],  'k--') #  random  predictions  curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False  Positive  Rate  or  (1  -  Specifity)') 
    plt.ylabel('True  Positive  Rate  or  (Sensitivity)') 
    plt.title('Receiver  Operating  Characteristic')


In [ ]:
score = resnet_model.evaluate(eval_x, eval_y, verbose = 1)

In [ ]:
import tensorflow as tf
output = resnet_model.predict(eval_x, verbose = 1)
np.argmax(output, axis = 1)
pred_y = np.array(output).flatten()

print(pred_y)

In [ ]:
print(f"Loss: {score[0]}\nAccuracy: {score[1]}")
evaluate_on_training_set(eval_y, pred_y.round())